**Masalah-Masalah dalam Melatih DNNs:**

* **Vanishing/Exploding Gradients:** Selama *backpropagation*, gradien bisa menjadi sangat kecil (vanishing gradients) atau sangat besar (exploding gradients) saat mengalir mundur melalui lapisan-lapisan bawah DNN. [cite_start]Kedua masalah ini membuat pelatihan lapisan bawah menjadi sangat sulit[cite: 6, 7, 8].
    * **Vanishing Gradients:** Gradien semakin mengecil saat algoritma bergerak menuju lapisan-lapisan bawah. [cite_start]Akibatnya, pembaruan bobot pada lapisan-lapun bawah hampir tidak berubah, dan pelatihan tidak pernah konvergen ke solusi yang baik[cite: 19, 20, 21].
    * **Exploding Gradients:** Kebalikannya bisa terjadi, di mana gradien menjadi semakin besar hingga lapisan mendapatkan pembaruan bobot yang sangat besar dan algoritma menjadi divergen. [cite_start]Ini sering muncul pada *recurrent neural networks* (RNNs)[cite: 21, 22].
    * **Penyebab:** Xavier Glorot dan Yoshua Bengio pada tahun 2010 menemukan bahwa kombinasi fungsi aktivasi sigmoid logistik populer dan teknik inisialisasi bobot standar (distribusi normal dengan mean 0 dan standard deviasi 1) berkontribusi pada masalah ini. [cite_start]Dengan kombinasi ini, varians output setiap lapisan jauh lebih besar dari varians inputnya, menyebabkan saturasi fungsi aktivasi di lapisan atas[cite: 25, 26, 27, 28]. [cite_start]Fungsi logistik memiliki mean 0.5 (bukan 0), memperburuk saturasi[cite: 29]. [cite_start]Saat input menjadi besar (negatif atau positif), fungsi sigmoid jenuh pada 0 atau 1, dengan turunan yang sangat mendekati 0, sehingga *backpropagation* hampir tidak memiliki gradien untuk disebarkan kembali[cite: 30, 31].

**Solusi untuk Masalah Gradien:**

1.  **Glorot dan He Initialization:**
    * [cite_start]**Prinsip:** Agar sinyal mengalir dengan baik ke dua arah (maju untuk prediksi dan mundur untuk *backpropagation*), varians output setiap lapisan harus sama dengan varians inputnya, dan gradien harus memiliki varians yang sama sebelum dan sesudah mengalir melalui lapisan secara terbalik[cite: 35, 36, 37].
    * [cite_start]**Glorot Initialization (Xavier Initialization):** Bobot koneksi setiap lapisan harus diinisialisasi secara acak menggunakan distribusi Normal dengan mean 0 dan varians $\sigma^{2}=\frac{1}{fan_{avg}}$ atau distribusi Uniform antara -r dan +r, dengan $r=\sqrt{\frac{3}{fan_{avg}}}$, di mana $fan_{avg}=(fan_{in}+fan_{out})/2$[cite: 38, 39, 43]. [cite_start]Ini sangat mempercepat pelatihan[cite: 46].
    * [cite_start]**LeCun Initialization:** Mirip dengan Glorot initialization, tetapi menggunakan $fan_{in}$ alih-alih $fan_{avg}$[cite: 44]. [cite_start]Ini setara dengan Glorot initialization ketika $fan_{in}=fan_{out}$[cite: 45]. [cite_start]Disarankan untuk fungsi aktivasi SELU[cite: 51, 53].
    * [cite_start]**He Initialization:** Inisialisasi untuk fungsi aktivasi ReLU dan variannya, menggunakan $\sigma^{2}=\frac{2}{fan_{in}}$[cite: 49, 53].
    * **Implementasi di Keras:**
        * [cite_start]Secara default, Keras menggunakan Glorot initialization dengan distribusi uniform[cite: 54].
        * [cite_start]Untuk He initialization: `keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")`[cite: 56].
        * Untuk He initialization dengan `fan_avg`:
            ```python
            he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                                            distribution='uniform')
            keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)
            ```
            [cite_start][cite: 58]

2.  **Nonsaturating Activation Functions:**
    * [cite_start]**Masalah dengan Sigmoid:** Fungsi aktivasi sigmoid jenuh untuk nilai input yang besar (positif atau negatif), menyebabkan gradien mendekati nol dan masalah *vanishing gradients*[cite: 30].
    * [cite_start]**ReLU (Rectified Linear Unit):** Tidak jenuh untuk nilai positif dan cepat dihitung[cite: 60].
        * **Masalah Dying ReLUs:** Beberapa neuron berhenti mengeluarkan apa pun selain 0 selama pelatihan, terutama dengan learning rate yang besar. [cite_start]Ini terjadi ketika jumlah bobot input neuron menjadi negatif untuk semua instance di set pelatihan, membuat gradien ReLU nol untuk input negatif[cite: 61, 62, 63].
    * **Leaky ReLU:** Didefinisikan sebagai $LeakyReLU_{\alpha}(z)=max(\alpha z,z)$. [cite_start]Memiliki kemiringan kecil ($\alpha$, biasanya 0.01 atau 0.2) untuk $z<0$, memastikan neuron tidak pernah mati sepenuhnya[cite: 64, 65, 66]. [cite_start]Varian Leaky ReLU lainnya termasuk Randomized Leaky ReLU (RReLU) dan Parametric Leaky ReLU (PReLU)[cite: 69, 71].
    * **ELU (Exponential Linear Unit):** Menawarkan kinerja yang lebih baik daripada varian ReLU. [cite_start]Memiliki nilai negatif untuk $z<0$ (membantu mengatasi *vanishing gradients* dengan rata-rata output mendekati 0), memiliki gradien nonzero untuk $z<0$ (menghindari masalah neuron mati), dan jika $\alpha=1$, fungsinya mulus di sekitar $z=0$ (mempercepat Gradient Descent)[cite: 75, 77, 80, 81, 83, 84]. [cite_start]Kekurangannya adalah lebih lambat dihitung karena fungsi eksponensial[cite: 85].
    * **SELU (Scaled ELU):** Varian skala dari ELU. [cite_start]Jika semua lapisan tersembunyi menggunakan SELU, jaringan akan *self-normalize* (output setiap lapisan akan cenderung mempertahankan mean 0 dan standar deviasi 1 selama pelatihan), menyelesaikan masalah *vanishing/exploding gradients*[cite: 88, 89].
        * [cite_start]**Syarat Self-Normalizing:** Fitur input harus distandardisasi (mean 0, standar deviasi 1)[cite: 91]. [cite_start]Bobot lapisan tersembunyi harus diinisialisasi dengan LeCun normal initialization (`kernel_initializer="lecun_normal"`)[cite: 92]. [cite_start]Arsitektur jaringan harus sekuensial[cite: 93].
    * [cite_start]**Pilihan Fungsi Aktivasi:** SELU > ELU > Leaky ReLU (dan variannya) > ReLU > tanh > logistic[cite: 97]. [cite_start]Jika arsitektur jaringan tidak memungkinkan *self-normalization*, ELU mungkin lebih baik dari SELU[cite: 98]. [cite_start]Jika latensi runtime penting, Leaky ReLU bisa menjadi pilihan[cite: 99].
    * **Implementasi di Keras:**
        * Leaky ReLU:
            ```python
            model = keras.models.Sequential([
                # ...
                keras.layers.Dense(10, kernel_initializer="he_normal"),
                keras.layers.LeakyReLU(alpha=0.2),
                # ...
            ])
            ```
            [cite_start][cite: 105]
        * [cite_start]SELU: `keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")` [cite: 106]

3.  **Batch Normalization (BN):**
    * **Konsep:** Menambahkan operasi di model tepat sebelum atau sesudah fungsi aktivasi setiap lapisan tersembunyi. [cite_start]Operasi ini men-*zero-center* dan menormalkan setiap input, kemudian menskalakan dan menggeser hasilnya menggunakan dua vektor parameter baru per lapisan (satu untuk penskalaan $\gamma$, satu untuk pergeseran $\beta$)[cite: 109, 110]. [cite_start]Ini memungkinkan model mempelajari skala dan mean optimal dari setiap input lapisan[cite: 111].
    * [cite_start]**Cara Kerja:** Selama pelatihan, BN memperkirakan mean ($\mu_B$) dan standar deviasi ($\sigma_B$) input dari *mini-batch* saat ini[cite: 114, 115]. [cite_start]Kemudian, input yang dinormalisasi dihitung: $\hat{x}^{(i)}=\frac{x^{(i)}-\mu_{B}}{\sqrt{\sigma_{B}}^{2}+\epsilon}}$[cite: 117]. [cite_start]Outputnya kemudian diskalakan dan digeser: $z^{(i)}=\gamma\otimes\hat{x}^{(i)}+\mathcal{\beta}$[cite: 117].
    * **Saat Testing:** Saat waktu pengujian, BN tidak dapat menghitung mean dan standar deviasi per *mini-batch* (karena mungkin hanya ada satu instance atau *batch* terlalu kecil). [cite_start]Solusinya adalah menggunakan rata-rata bergerak (moving average) dari mean dan standar deviasi input lapisan yang dihitung selama pelatihan[cite: 129, 133, 134].
    * [cite_start]**Keuntungan:** Mengurangi masalah *vanishing gradients* secara signifikan, bahkan memungkinkan penggunaan fungsi aktivasi saturasi seperti tanh dan sigmoid[cite: 138]. [cite_start]Membuat jaringan kurang sensitif terhadap inisialisasi bobot[cite: 139]. [cite_start]Memungkinkan *learning rate* yang lebih besar, mempercepat proses pembelajaran[cite: 140]. [cite_start]Bertindak sebagai regularizer, mengurangi kebutuhan akan teknik regularisasi lainnya[cite: 144].
    * [cite_start]**Kekurangan:** Menambah kompleksitas pada model[cite: 145]. [cite_start]Ada penalti waktu runtime karena komputasi ekstra di setiap lapisan, meskipun ini dapat dihindari setelah pelatihan dengan menggabungkan lapisan BN dengan lapisan sebelumnya[cite: 146, 147].
    * **Implementasi di Keras:**
        * BN setelah lapisan tersembunyi:
            ```python
            model = keras.models.Sequential([
                keras.layers.Flatten(input_shape=[28, 28]),
                keras.layers.BatchNormalization(), # BN sebagai lapisan pertama
                keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(10, activation="softmax")
            ])
            ```
            [cite_start][cite: 157]
        * BN sebelum fungsi aktivasi (dan menghilangkan bias pada lapisan sebelumnya):
            ```python
            model = keras.models.Sequential([
                keras.layers.Flatten(input_shape=[28, 28]),
                keras.layers.BatchNormalization(),
                keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.Activation("elu"),
                keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
                keras.layers.BatchNormalization(),
                keras.layers.Activation("elu"),
                keras.layers.Dense(10, activation="softmax")
            ])
            ```
            [cite_start][cite: 170]
        * [cite_start]Setiap lapisan BN menambahkan empat parameter per input: $\gamma, \beta, \mu, \sigma$[cite: 161]. [cite_start]$\gamma$ dan $\beta$ dapat dilatih (*trainable*), sedangkan $\mu$ dan $\sigma$ (rata-rata bergerak) tidak dilatih oleh *backpropagation*[cite: 162, 163].
        * [cite_start]Hyperparameter `momentum` (default 0.9, 0.99, atau 0.999) digunakan untuk memperbarui rata-rata bergerak[cite: 173, 174, 175].
        * [cite_start]Hyperparameter `axis` (default -1) menentukan sumbu mana yang harus dinormalisasi[cite: 176, 177].

4.  **Gradient Clipping:**
    * [cite_start]**Konsep:** Memotong gradien selama *backpropagation* sehingga tidak pernah melebihi ambang batas tertentu[cite: 192, 193]. [cite_start]Paling sering digunakan dalam RNNs[cite: 193].
    * **Implementasi di Keras:** Atur argumen `clipvalue` atau `clipnorm` saat membuat optimizer.
        * [cite_start]`clipvalue=1.0`: Memotong setiap komponen vektor gradien antara -1.0 dan 1.0[cite: 194, 195]. [cite_start]Dapat mengubah orientasi vektor gradien[cite: 196, 197, 198].
        * [cite_start]`clipnorm=1.0`: Memotong seluruh gradien jika norma $l_2$-nya lebih besar dari ambang batas, mempertahankan orientasi vektor gradien[cite: 199, 200, 201].
        * [cite_start]Contoh: `optimizer = keras.optimizers.SGD(clipvalue=1.0)` [cite: 194]

**Optimasi Lebih Cepat:**

[cite_start]Selain teknik di atas, penggunaan optimizer yang lebih cepat dari Gradient Descent dapat mempercepat pelatihan[cite: 279].

1.  **Momentum Optimization:**
    * **Konsep:** Mirip bola bowling yang menggelinding menuruni lereng. Mempertimbangkan gradien sebelumnya melalui vektor momentum `m`, yang merupakan jumlah eksponensial gradien masa lalu. [cite_start]Gradien digunakan untuk akselerasi, bukan kecepatan[cite: 282, 288, 289].
    * **Algoritma:**
        1.  $m\leftarrow\beta m-\eta\nabla_{\theta}J(\theta)$
        2.  $\theta\leftarrow\theta+m$
        [cite_start]Di mana $\beta$ adalah hyperparameter momentum (biasanya 0.9) dan $\eta$ adalah *learning rate*[cite: 290, 291].
    * [cite_start]**Keuntungan:** Mencapai kecepatan terminal lebih cepat (misalnya, 10x lebih cepat dari GD jika $\beta=0.9$)[cite: 292]. [cite_start]Membantu keluar dari *plateaus* dan bergerak lebih cepat di lembah yang memanjang[cite: 293, 296, 298].
    * [cite_start]**Implementasi di Keras:** `optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)` [cite: 302]

2.  **Nesterov Accelerated Gradient (NAG):**
    * **Konsep:** Varian dari Momentum Optimization yang hampir selalu lebih cepat. [cite_start]Mengukur gradien fungsi biaya sedikit di depan, ke arah momentum (pada $\theta+\beta m$)[cite: 302, 303].
    * **Algoritma:**
        1.  $m\leftarrow\beta m-\eta\nabla_{\theta}J(\theta+\beta m)$
        2.  $\theta\leftarrow\theta+m$
        [cite_start][cite: 303]
    * [cite_start]**Keuntungan:** Lebih akurat karena momentum umumnya mengarah ke optimum, membantu mengurangi osilasi dan konvergen lebih cepat[cite: 304, 305, 306, 307].
    * [cite_start]**Implementasi di Keras:** `optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)` [cite: 308]

3.  **AdaGrad:**
    * [cite_start]**Konsep:** Menurunkan skala vektor gradien di sepanjang dimensi yang paling curam (memiliki *adaptive learning rate*)[cite: 312, 313, 319, 320].
    * **Algoritma:**
        1.  [cite_start]$s\leftarrow s+\nabla_{\theta}J(\theta)\otimes\nabla_{\theta}J(\theta)$ (mengakumulasi kuadrat gradien) [cite: 314]
        2.  [cite_start]$\theta\leftarrow\theta-\eta\nabla_{\theta}J(\theta)\oslash\sqrt{s+\epsilon}$ (menurunkan skala gradien) [cite: 317]
    * [cite_start]**Keuntungan:** Membutuhkan lebih sedikit penyetelan *learning rate*[cite: 321].
    * [cite_start]**Kekurangan:** Sering berhenti terlalu dini saat melatih jaringan saraf karena *learning rate* menjadi terlalu kecil[cite: 323, 324]. [cite_start]Umumnya tidak disarankan untuk DNNs[cite: 324].

4.  **RMSProp:**
    * [cite_start]**Konsep:** Memperbaiki masalah AdaGrad dengan hanya mengakumulasi gradien dari iterasi terbaru menggunakan *exponential decay*[cite: 327, 328].
    * **Algoritma:**
        1.  [cite_start]$s\leftarrow\beta s+(1-\beta)\nabla_{\theta}J(\theta)\otimes\nabla_{\theta}J(\theta)$ (decay rate $\beta$ biasanya 0.9) [cite: 329]
        2.  $\theta\leftarrow\theta-\eta\nabla_{\theta}J(\theta)\oslash\sqrt{s+\epsilon}$
        [cite_start][cite: 329]
    * [cite_start]**Keuntungan:** Kinerja jauh lebih baik daripada AdaGrad kecuali pada masalah yang sangat sederhana[cite: 333].
    * [cite_start]**Implementasi di Keras:** `optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)` [cite: 332]

5.  **Adam (Adaptive Moment Estimation):**
    * [cite_start]**Konsep:** Menggabungkan ide Momentum Optimization (rata-rata bergerak gradien masa lalu) dan RMSProp (rata-rata bergerak kuadrat gradien masa lalu)[cite: 335].
    * **Algoritma:**
        1.  [cite_start]$m\leftarrow\beta_{1}m-(1-\beta_{1})\nabla_{\theta}J(\theta)$ (rata-rata bergerak gradien) [cite: 338]
        2.  [cite_start]$s\leftarrow\beta_{2}s+(1-\beta_{2})\nabla_{\theta}J(\theta)\otimes\nabla_{\theta}J(\theta)$ (rata-rata bergerak kuadrat gradien) [cite: 338]
        3.  [cite_start]$\hat{m}\leftarrow\frac{m}{1-\beta_{1}^{t}}$ (koreksi bias) [cite: 338, 341]
        4.  [cite_start]$\hat{s}\leftarrow\frac{s}{1-\beta_{2}^{t}}$ (koreksi bias) [cite: 338, 341]
        5.  [cite_start]$\theta\leftarrow\theta+\eta~m\oslash\sqrt{s+\epsilon}$ (pembaruan parameter) [cite: 338]
    * [cite_start]**Hyperparameter:** $\beta_1$ (momentum decay) biasanya 0.9, $\beta_2$ (scaling decay) biasanya 0.999, $\epsilon$ (smoothing term) biasanya $10^{-7}$[cite: 342, 343, 344].
    * [cite_start]**Keuntungan:** Algoritma *adaptive learning rate* yang membutuhkan lebih sedikit penyetelan *learning rate* dan seringkali dapat menggunakan nilai default $\eta=0.001$[cite: 345, 346].
    * [cite_start]**Implementasi di Keras:** `optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)` [cite: 345]

6.  **Nadam:**
    * [cite_start]**Konsep:** Adam Optimization ditambah trik Nesterov (Nesterov Accelerated Momentum)[cite: 355].
    * [cite_start]**Keuntungan:** Seringkali konvergen sedikit lebih cepat daripada Adam[cite: 355, 356].
    * **Catatan:** Metode optimasi adaptif (RMSProp, Adam, Nadam) umumnya bagus dan konvergen cepat, tetapi dapat menghasilkan solusi yang kurang generalisasi pada beberapa dataset. [cite_start]Dalam kasus tersebut, Nesterov Accelerated Gradient mungkin lebih baik[cite: 357, 358, 359, 360].

**Training Sparse Models:**

* [cite_start]**Tujuan:** Untuk model yang sangat cepat saat runtime atau membutuhkan memori lebih sedikit[cite: 368].
* **Cara:**
    1.  [cite_start]Latih model seperti biasa, lalu singkirkan bobot yang sangat kecil (setel menjadi nol)[cite: 369]. [cite_start]Ini mungkin tidak menghasilkan model yang sangat jarang dan dapat menurunkan kinerja[cite: 370].
    2.  [cite_start]Terapkan regularisasi $l_1$ yang kuat selama pelatihan, yang mendorong optimizer untuk men-*zero-out* bobot sebanyak mungkin[cite: 371].
    3.  [cite_start]Gunakan TensorFlow Model Optimization Toolkit (TF-MOT) yang menyediakan API *pruning* untuk menghapus koneksi secara iteratif selama pelatihan berdasarkan magnitudenya[cite: 372].

**Learning Rate Scheduling:**

* [cite_start]**Tujuan:** Mengurangi *learning rate* selama pelatihan untuk mencapai solusi yang lebih baik dan lebih cepat[cite: 385, 386].
* **Jenis:**
    1.  **Power Scheduling:** $\eta(t)=\eta_{0}/(1+t/s)^{c}$. [cite_start]*Learning rate* menurun pada setiap langkah, awalnya cepat lalu melambat[cite: 389, 390, 391, 392].
        * [cite_start]**Implementasi di Keras:** Atur hyperparameter `decay` saat membuat optimizer: `optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)`[cite: 409].
    2.  **Exponential Scheduling:** $\eta(t)=\eta_{0}0.1^{t/s}$. [cite_start]*Learning rate* secara bertahap menurun dengan faktor 10 setiap `s` langkah[cite: 393, 394].
        * **Implementasi di Keras:** Gunakan `keras.callbacks.LearningRateScheduler`.
            ```python
            def exponential_decay_fn(epoch):
                return 0.01 * 0.1**(epoch / 20)
            lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
            history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])
            ```
            [cite_start][cite: 410, 411, 412]
        * Alternatif dengan `keras.optimizers.schedules`:
            ```python
            s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size=32)
            learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
            optimizer = keras.optimizers.SGD(learning_rate)
            ```
            [cite_start][cite: 429]
    3.  [cite_start]**Piecewise Constant Scheduling:** Menggunakan *learning rate* konstan untuk sejumlah *epoch*, lalu *learning rate* yang lebih kecil untuk *epoch* berikutnya, dan seterusnya[cite: 396].
        * **Implementasi di Keras:** Gunakan `LearningRateScheduler` dengan fungsi kustom:
            ```python
            def piecewise_constant_fn(epoch):
                if epoch < 5:
                    return 0.01
                elif epoch < 15:
                    return 0.005
                else:
                    return 0.001
            ```
            [cite_start][cite: 424]
    4.  [cite_start]**Performance Scheduling:** Mengukur *validation error* setiap N langkah dan mengurangi *learning rate* ketika *error* berhenti menurun[cite: 398, 399].
        * [cite_start]**Implementasi di Keras:** Gunakan `ReduceLROnPlateau` callback: `lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)`[cite: 425, 426].
    5.  [cite_start]**1cycle Scheduling:** Meningkatkan *learning rate* secara linear dari $\eta_0$ ke $\eta_1$ di paruh pertama pelatihan, lalu menurunkannya secara linear kembali ke $\eta_0$ di paruh kedua, dan diakhiri dengan penurunan tajam[cite: 399, 400, 401]. [cite_start]Momentum juga divariasikan[cite: 402]. [cite_start]Ini seringkali mempercepat pelatihan dan mencapai kinerja yang lebih baik[cite: 403].

**Menghindari Overfitting Melalui Regularisasi:**

Selain *early stopping* dan *Batch Normalization*, ada teknik regularisasi lain:

1.  **$l_1$ dan $l_2$ Regularization:**
    * [cite_start]**Konsep:** Menambahkan *term* *regularization loss* ke fungsi *loss* keseluruhan untuk membatasi bobot koneksi jaringan saraf[cite: 442].
    * [cite_start]**$l_2$ Regularization (Weight Decay):** Mendorong bobot menjadi kecil[cite: 442].
    * [cite_start]**$l_1$ Regularization:** Mendorong banyak bobot menjadi nol, menghasilkan model yang jarang (*sparse model*)[cite: 442].
    * **Implementasi di Keras:** Gunakan `kernel_regularizer` saat membuat lapisan.
        * [cite_start]$l_2$: `kernel_regularizer=keras.regularizers.l2(0.01)` [cite: 443]
        * [cite_start]$l_1$: `keras.regularizers.l1()` [cite: 445]
        * [cite_start]$l_1$ dan $l_2$: `keras.regularizers.l1_l2()` [cite: 446]
        * [cite_start]Untuk mengurangi pengulangan argumen, dapat menggunakan `functools.partial()`[cite: 447, 449]:
            ```python
            from functools import partial
            RegularizedDense = partial(keras.layers.Dense,
                                     activation="elu",
                                     kernel_initializer="he_normal",
                                     kernel_regularizer=keras.regularizers.l2(0.01))
            model = keras.models.Sequential([
                keras.layers.Flatten(input_shape=[28, 28]),
                RegularizedDense(300),
                RegularizedDense(100),
                RegularizedDense(10, activation="softmax",
                                 kernel_initializer="glorot_uniform")
            ])
            ```
            [cite_start][cite: 449, 450]

2.  **Dropout:**
    * [cite_start]**Konsep:** Pada setiap langkah pelatihan, setiap neuron (kecuali neuron output) memiliki probabilitas `p` (dropout rate) untuk dinonaktifkan sementara (output 0)[cite: 453, 454]. [cite_start]Setelah pelatihan, neuron tidak dinonaktifkan lagi[cite: 456].
    * [cite_start]**Dropout Rate:** Biasanya antara 10% dan 50% (20-30% untuk RNNs, 40-50% untuk CNNs)[cite: 455].
    * **Mekanisme:** Neuron yang dilatih dengan dropout tidak dapat beradaptasi bersama dengan neuron tetangganya; mereka harus berfungsi secara mandiri dan tidak dapat terlalu bergantung pada beberapa input neuron. [cite_start]Ini menghasilkan jaringan yang lebih kuat dan menggeneralisasi lebih baik[cite: 467, 468, 469, 470, 471, 472]. [cite_start]Ini juga dapat dilihat sebagai *averaging ensemble* dari banyak jaringan saraf yang lebih kecil[cite: 478].
    * **Penting:** Selama pengujian, neuron tidak di-*drop*. [cite_start]Untuk mengkompensasi, bobot koneksi input setiap neuron perlu dikalikan dengan `(1-p)` setelah pelatihan (atau output setiap neuron dibagi dengan `(1-p)` selama pelatihan)[cite: 481, 482, 484, 485].
    * **Implementasi di Keras:** Gunakan `keras.layers.Dropout` layer.
        ```python
        model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=[28, 28]),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
            keras.layers.Dropout(rate=0.2),
            keras.layers.Dense(10, activation="softmax")
        ])
        ```
        [cite_start][cite: 490]
    * [cite_start]**Catatan:** Dropout memperlambat konvergensi, tetapi biasanya menghasilkan model yang jauh lebih baik[cite: 493, 494].
    * [cite_start]**Alpha Dropout:** Untuk jaringan *self-normalizing* dengan fungsi aktivasi SELU, gunakan Alpha Dropout, yang mempertahankan mean dan standar deviasi inputnya[cite: 495].

3.  **Monte Carlo (MC) Dropout:**
    * **Konsep:** Teknik pasca-pelatihan yang dapat meningkatkan kinerja model dropout yang sudah terlatih tanpa pelatihan ulang atau modifikasi. [cite_start]Ini juga memberikan ukuran ketidakpastian model yang lebih baik[cite: 497, 498].
    * **Cara Kerja:** Buat beberapa prediksi (misalnya 100) pada set uji, dengan mengaktifkan lapisan Dropout (mengatur `training=True`). Karena Dropout aktif, semua prediksi akan berbeda. [cite_start]Rata-rata dari prediksi-prediksi ini memberikan estimasi Monte Carlo yang lebih andal daripada satu prediksi tanpa Dropout[cite: 499, 500, 501, 508].
    * [cite_start]**Keuntungan:** Peningkatan akurasi model, estimasi probabilitas yang lebih andal, dan informasi ketidakpastian (melalui standar deviasi prediksi)[cite: 508, 513, 514, 515, 516].
    * **Implementasi (contoh):**
        ```python
        import numpy as np
        # Misalkan model sudah dilatih
        # y_test adalah label sebenarnya
        # X_test_scaled adalah data uji yang sudah diskalakan

        y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
        y_proba = y_probas.mean(axis=0)
        y_pred = np.argmax(y_proba, axis=1) # Prediksi akhir
        accuracy = np.sum(y_pred == y_test.flatten()) / len(y_test) # Hitung akurasi
        print(f"MC Dropout accuracy: {accuracy}")
        ```
        [cite_start][cite: 499, 500, 507, 516]
    * Jika model berisi lapisan lain yang berperilaku berbeda selama pelatihan (seperti BatchNormalization), perlu subclass `Dropout` menjadi `MCDropout` untuk memaksa `training=True` saat inferensi:
        ```python
        class MCDropout(keras.layers.Dropout):
            def call(self, inputs):
                return super().call(inputs, training=True)
        ```
        [cite_start][cite: 523, 524]

4.  **Max-Norm Regularization:**
    * [cite_start]**Konsep:** Membatasi bobot `w` koneksi yang masuk ke setiap neuron sehingga norma $l_2$-nya tidak melebihi hyperparameter `r` ($||w||_2 \le r$)[cite: 529, 530].
    * [cite_start]**Cara Kerja:** Setelah setiap langkah pelatihan, $||w||_2$ dihitung dan `w` diskalakan ulang jika diperlukan ($w \leftarrow w \cdot r / ||w||_2$)[cite: 532].
    * [cite_start]**Keuntungan:** Mengurangi overfitting, dapat membantu mengatasi masalah gradien tidak stabil jika BN tidak digunakan[cite: 535, 536].
    * **Implementasi di Keras:** Setel `kernel_constraint` argumen setiap lapisan tersembunyi ke `keras.constraints.max_norm(r)`:
        ```python
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))
        ```
        [cite_start][cite: 538]

**Panduan Praktis (Ringkasan):**

* [cite_start]**Inisialisasi Kernel:** He initialization (default)[cite: 545]. [cite_start]Untuk jaringan *self-normalizing*, gunakan LeCun initialization[cite: 549].
* [cite_start]**Fungsi Aktivasi:** ELU (default)[cite: 545]. [cite_start]Untuk jaringan *self-normalizing*, gunakan SELU[cite: 549].
* [cite_start]**Normalisasi:** Tidak ada jika jaringan dangkal, Batch Normalization jika jaringan dalam[cite: 545]. [cite_start]Untuk jaringan *self-normalizing*, tidak ada (self-normalization)[cite: 549].
* [cite_start]**Regularisasi:** *Early stopping* (+ $l_2$ regularization jika diperlukan)[cite: 545]. [cite_start]Untuk jaringan *self-normalizing*, Alpha Dropout jika diperlukan[cite: 549].
* [cite_start]**Optimizer:** Momentum optimization (atau RMSProp atau Nadam)[cite: 545, 549].
* [cite_start]**Learning Rate Schedule:** 1cycle[cite: 545, 549].
* [cite_start]**Normalisasi Input:** Selalu normalisasi fitur input (mean 0, standar deviasi 1)[cite: 550].
* **Transfer Learning & Pretraining:**
    * [cite_start]**Transfer Learning:** Gunakan kembali bagian dari jaringan saraf yang sudah dilatih sebelumnya pada tugas serupa untuk mempercepat pelatihan dan mengurangi data yang dibutuhkan[cite: 204, 206]. [cite_start]Ganti lapisan output, bekukan lapisan yang digunakan kembali di awal, lalu *unfreeze* dan sesuaikan dengan *learning rate* yang lebih rendah[cite: 213, 218, 219, 221].
    * [cite_start]**Unsupervised Pretraining:** Latih model tak terawasi (seperti *autoencoder* atau GAN) pada data tak berlabel yang banyak, lalu gunakan kembali lapisan bawah untuk tugas yang sebenarnya dengan data berlabel[cite: 252, 254, 255].
    * [cite_start]**Pretraining on an Auxiliary Task:** Latih jaringan saraf pertama pada tugas tambahan yang data berlabelnya mudah diperoleh, lalu gunakan kembali lapisan bawahnya untuk tugas utama[cite: 265, 266].
* **Kasus Khusus:**
    * [cite_start]**Sparse Model:** Gunakan $l_1$ regularization atau TensorFlow Model Optimization Toolkit (akan membatalkan *self-normalization*)[cite: 551, 552, 553, 554].
    * [cite_start]**Low-Latency Model:** Kurangi jumlah lapisan, gabungkan lapisan Batch Normalization, gunakan fungsi aktivasi yang lebih cepat (Leaky ReLU atau ReLU), gunakan model *sparse*, dan pertimbangkan presisi *float* yang lebih rendah[cite: 555, 556].
    * [cite_start]**Risk-Sensitive Application / Ketidakpastian Penting:** Gunakan MC Dropout untuk meningkatkan kinerja dan mendapatkan estimasi probabilitas yang lebih andal bersama dengan estimasi ketidakpastian[cite: 557].

**Latihan (dan Penjelasan Teoritis):**

1.  **Apakah boleh menginisialisasi semua bobot ke nilai yang sama asalkan nilai tersebut dipilih secara acak menggunakan He initialization?**
    * **Tidak.** Menginisialisasi semua bobot ke nilai yang sama, bahkan jika nilai itu dipilih secara acak menggunakan He initialization, adalah ide yang buruk. Jika semua bobot diinisialisasi ke nilai yang sama, maka semua neuron dalam satu lapisan akan memiliki input yang identik, dan akibatnya, mereka akan menghasilkan output yang identik. Selama *backpropagation*, semua neuron di lapisan tersebut akan menerima gradien yang identik dan akan memperbarui bobot mereka dengan cara yang sama persis. Ini berarti semua neuron di lapisan tersebut akan tetap simetris, dan tidak akan ada yang belajar fitur yang berbeda. Jaringan tidak akan dapat belajar representasi yang kaya dan kompleks dari data, sehingga membatasi kemampuannya secara drastis.

2.  **Apakah boleh menginisialisasi *bias terms* ke 0?**
    * **Ya, umumnya boleh.** Menginisialisasi *bias terms* ke 0 adalah praktik umum dan biasanya tidak menimbulkan masalah. *Bias terms* memiliki efek yang berbeda dari bobot; mereka memungkinkan setiap neuron untuk menggeser fungsi aktivasinya, terlepas dari inputnya. Karena mereka tidak mengalami masalah simetri yang sama seperti bobot (yaitu, neuron akan tetap berbeda meskipun biasnya sama, karena bobotnya akan berbeda), menginisialisasi mereka ke nol biasanya merupakan titik awal yang aman dan efektif.

3.  **Sebutkan tiga keuntungan fungsi aktivasi SELU dibandingkan ReLU.**
    1.  **Self-Normalizing:** Jaringan yang hanya terdiri dari tumpukan lapisan *dense* dan menggunakan fungsi aktivasi SELU di semua lapisan tersembunyi akan *self-normalize*. [cite_start]Ini berarti output setiap lapisan akan cenderung mempertahankan mean 0 dan standar deviasi 1 selama pelatihan, secara efektif mengatasi masalah *vanishing/exploding gradients*[cite: 88].
    2.  [cite_start]**Menghindari Dying ReLUs:** Seperti ELU, SELU memiliki gradien nonzero untuk nilai input negatif, yang mencegah neuron "mati" dan berhenti belajar[cite: 83, 84]. [cite_start]ReLU standar menderita masalah "dying ReLUs" di mana neuron bisa berhenti aktif sama sekali[cite: 61, 63].
    3.  [cite_start]**Konvergensi Lebih Cepat:** Dengan *self-normalization*, SELU seringkali dapat mempercepat pelatihan secara signifikan dan menghasilkan kinerja yang lebih baik pada *test set*, terutama untuk jaringan yang sangat dalam[cite: 75, 76, 89].

4.  **Dalam kasus apa Anda ingin menggunakan setiap fungsi aktivasi berikut: SELU, Leaky ReLU (dan variannya), ReLU, tanh, *logistic*, dan *softmax*?**
    * **SELU:** Pilihan terbaik untuk jaringan *feedforward* yang dalam (lapisan *dense* berurutan) yang dapat *self-normalize*. [cite_start]Ini sering mengungguli fungsi aktivasi lainnya, terutama untuk *deep networks*, asalkan fitur input distandardisasi dan inisialisasi LeCun normal digunakan[cite: 89, 90, 91, 92, 97].
    * **ELU (Exponential Linear Unit):** Pilihan yang sangat baik secara umum jika *self-normalization* tidak dapat dijamin (misalnya, pada arsitektur non-sekuensial seperti RNN atau jaringan dengan *skip connections*), atau jika SELU tidak menunjukkan kinerja terbaik. [cite_start]Ini membantu mengurangi masalah *vanishing gradients* dan menghindari *dying ReLUs*, serta mulus di sekitar $z=0$[cite: 80, 81, 83, 84, 98].
    * **Leaky ReLU (dan variannya seperti RReLU, PReLU):** Pilihan yang baik jika Anda peduli dengan latensi runtime (karena lebih cepat dihitung daripada ELU/SELU) atau jika Anda mengalami masalah *dying ReLUs* dengan ReLU. [cite_start]Leaky ReLU selalu mengungguli ReLU murni[cite: 67, 99]. [cite_start]PReLU dapat menjadi pilihan jika Anda memiliki dataset pelatihan yang sangat besar dan ingin mengizinkan $\alpha$ dipelajari[cite: 71, 73].
    * **ReLU (Rectified Linear Unit):** Masih merupakan pilihan yang sangat umum karena cepat dihitung dan banyak pustaka serta akselerator *hardware* menyediakan optimasi khusus ReLU. [cite_start]Ini pilihan yang baik jika kecepatan adalah prioritas utama dan masalah *dying ReLUs* dapat dikelola (misalnya, dengan *learning rate* yang lebih kecil atau Batch Normalization)[cite: 60, 102, 103].
    * **tanh (Hyperbolic Tangent):** Fungsi aktivasi sigmoid yang men-*zero-center* outputnya (mean 0). Ini biasanya berkinerja sedikit lebih baik daripada fungsi *logistic* di jaringan yang dalam karena output yang *zero-centered* membantu dalam *backpropagation*. [cite_start]Namun, masih menderita masalah *vanishing gradients*[cite: 29, 97]. Cocok untuk lapisan tersembunyi jika Anda tidak bisa menggunakan ReLU atau variannya, atau jika Anda menggunakan Batch Normalization yang dapat mengatasi masalah saturasi.
    * [cite_start]**Logistic (Sigmoid):** Jarang digunakan untuk lapisan tersembunyi di DNNs yang dalam karena rentan terhadap masalah *vanishing gradients* akibat saturasi[cite: 26, 30, 97]. [cite_start]Paling cocok untuk **lapisan output** dalam kasus klasifikasi biner, di mana ia menghasilkan probabilitas antara 0 dan 1[cite: 228].
    * [cite_start]**Softmax:** Digunakan secara eksklusif untuk **lapisan output** dalam kasus klasifikasi multi-kelas, di mana ia menghasilkan distribusi probabilitas di antara kelas-kelas[cite: 574].

5.  **Apa yang mungkin terjadi jika Anda mengatur hyperparameter momentum terlalu dekat ke 1 (misalnya, 0.99999) saat menggunakan SGD optimizer?**
    * [cite_start]Jika hyperparameter momentum ($\beta$) diatur terlalu dekat ke 1 (misalnya, 0.99999), itu berarti *friction* dalam sistem sangat rendah[cite: 290]. [cite_start]Ini akan menyebabkan momentum menjadi terlalu besar dan optimasi akan **melampaui (*overshoot*) minimum** secara drastis, berosilasi dengan liar, dan **kesulitan untuk stabil atau konvergen** ke solusi yang baik[cite: 299, 300]. Pada dasarnya, optimasi akan memiliki terlalu banyak "inersia" dan akan kesulitan untuk berhenti atau membalikkan arah bahkan ketika gradien menunjukkan bahwa ia telah melewati optimum. Ini dapat menyebabkan pelatihan menjadi tidak stabil dan bahkan divergen.

6.  **Sebutkan tiga cara Anda dapat menghasilkan model yang *sparse*.**
    1.  **Menerapkan Regularisasi $l_1$ yang Kuat selama Pelatihan:** Regularisasi $l_1$ menambahkan *penalty* pada jumlah absolut bobot, mendorong optimizer untuk mengatur banyak bobot menjadi nol. [cite_start]Ini secara inheren menghasilkan model yang *sparse*[cite: 371].
    2.  **Memangkas (*Pruning*) Bobot Kecil setelah Pelatihan:** Latih model seperti biasa hingga konvergen, lalu identifikasi bobot yang sangat kecil dan atur nilainya menjadi nol. [cite_start]Meskipun metode ini sederhana, seringkali tidak menghasilkan model yang sangat *sparse* dan dapat sedikit menurunkan kinerja[cite: 369, 370].
    3.  **Menggunakan TensorFlow Model Optimization Toolkit (TF-MOT):** Toolkit ini menyediakan API *pruning* yang mampu secara iteratif menghapus koneksi (bobot) selama pelatihan berdasarkan magnitudenya. [cite_start]Ini adalah pendekatan yang lebih canggih dan efektif untuk menghasilkan model yang *sparse* dibandingkan hanya memangkas setelah pelatihan[cite: 372].

7.  **Apakah *dropout* memperlambat pelatihan? Apakah memperlambat inferensi (yaitu, membuat prediksi pada instance baru)? Bagaimana dengan MC Dropout?**
    * [cite_start]**Apakah *dropout* memperlambat pelatihan?** Ya, *dropout* cenderung **secara signifikan memperlambat konvergensi pelatihan**[cite: 493]. Hal ini karena pada setiap langkah pelatihan, subset neuron yang berbeda di-*drop out*, yang berarti jaringan harus beradaptasi dan tidak dapat sepenuhnya mengandalkan ko-adaptasi antar neuron. Setiap langkah pelatihan pada dasarnya melatih jaringan yang unik, yang menambah variabilitas dan memperlambat proses pembelajaran keseluruhan.
    * **Apakah memperlambat inferensi?** **Tidak, *dropout* tidak memperlambat inferensi.** Setelah pelatihan, neuron tidak lagi di-*drop*. Sebaliknya, bobot koneksi input setiap neuron diskalakan dengan faktor `(1-p)` (di mana `p` adalah *dropout rate*), atau output neuron dibagi dengan `(1-p)` selama pelatihan. [cite_start]Dengan demikian, selama inferensi, tidak ada komputasi tambahan yang dilakukan karena *dropout*, sehingga tidak ada penalti runtime[cite: 456, 488].
    * [cite_start]**Bagaimana dengan MC Dropout?** **Ya, MC Dropout memperlambat inferensi.** MC Dropout melibatkan pembuatan banyak prediksi (misalnya, 100 prediksi) pada *test set* dengan mengaktifkan *dropout* (`training=True`) selama fase inferensi[cite: 499, 500]. Karena setiap prediksi adalah hasil dari jaringan yang di-*drop* secara acak, ini secara efektif berarti menjalankan inferensi model berkali-kali. [cite_start]Oleh karena itu, waktu inferensi akan dikalikan dengan jumlah sampel Monte Carlo yang diambil[cite: 519].

8.  **Latihan melatih *deep neural network* pada dataset gambar CIFAR10:**
    Ini adalah latihan praktis yang melibatkan implementasi kode. Saya tidak bisa menjalankan kode atau memberikan output hasil pelatihan, tetapi saya bisa memberikan panduan teoritis dan struktur kode yang diperlukan untuk setiap sub-bagian.

    **a. Bangun DNN dengan 20 *hidden layers* masing-masing 100 neuron. Gunakan He initialization dan fungsi aktivasi ELU.**
    * [cite_start]**Teori:** Untuk jaringan *deep* seperti ini, He initialization dan ELU adalah pilihan yang direkomendasikan untuk mengatasi masalah *vanishing/exploding gradients* dan *dying ReLUs*[cite: 49, 60, 80]. ELU membantu menjaga mean output mendekati nol dan memiliki gradien nonzero untuk input negatif.
    * **Struktur Kode:**
        ```python
        import tensorflow as tf
        from tensorflow import keras

        model = keras.models.Sequential()
        model.add(keras.layers.Flatten(input_shape=[32, 32, 3])) # CIFAR10 images are 32x32 pixels with 3 color channels
        for _ in range(20):
            model.add(keras.layers.Dense(100, activation="elu",
                                         kernel_initializer="he_normal"))
        model.add(keras.layers.Dense(10, activation="softmax")) # 10 classes for CIFAR10
        ```

    **b. Menggunakan Nadam optimization dan *early stopping*, latih jaringan pada dataset CIFAR10. Anda dapat memuatnya dengan `keras.datasets.cifar10.load_data()`. Dataset ini terdiri dari 60.000 gambar warna $32\times32$-piksel (50.000 untuk pelatihan, 10.000 untuk pengujian) dengan 10 kelas, jadi Anda akan membutuhkan *softmax output layer* dengan 10 neuron. Ingatlah untuk mencari *learning rate* yang tepat setiap kali Anda mengubah arsitektur model atau *hyperparameter*.**
    * [cite_start]**Teori:** Nadam adalah optimizer yang kuat yang menggabungkan Momentum dan RMSProp dengan trik Nesterov, seringkali memberikan konvergensi yang cepat[cite: 355, 356]. [cite_start]*Early stopping* adalah teknik regularisasi yang mencegah *overfitting* dengan menghentikan pelatihan ketika kinerja pada set validasi berhenti membaik[cite: 439]. [cite_start]Menemukan *learning rate* yang optimal sangat penting untuk pelatihan yang efisien[cite: 376].
    * **Struktur Kode:**
        ```python
        (X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

        # Normalisasi input (penting!)
        X_train_full = X_train_full / 255.0
        X_test = X_test / 255.0

        # Membagi data menjadi training dan validation
        X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
        y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

        # Mengkonversi label menjadi one-hot encoding (jika loss function membutuhkan)
        # y_train = keras.utils.to_categorical(y_train, 10)
        # y_valid = keras.utils.to_categorical(y_valid, 10)
        # y_test = keras.utils.to_categorical(y_test, 10)

        # Cari learning rate yang tepat (optional, tapi disarankan)
        # lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))
        # optimizer = keras.optimizers.Nadam(learning_rate=1e-4) # Start with small LR
        # model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        # history = model.fit(X_train, y_train, epochs=100, callbacks=[lr_scheduler])
        # Plot loss vs LR untuk menemukan LR terbaik

        # Setelah menemukan LR terbaik, inisialisasi ulang model dan latih
        optimizer = keras.optimizers.Nadam(learning_rate=0.001) # Contoh LR terbaik
        model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

        history = model.fit(X_train, y_train, epochs=100,
                            validation_data=(X_valid, y_valid),
                            callbacks=[early_stopping_cb])
        ```

    **c. Sekarang coba tambahkan Batch Normalization dan bandingkan *learning curves*: Apakah konvergensi lebih cepat dari sebelumnya? Apakah menghasilkan model yang lebih baik? Bagaimana pengaruhnya terhadap kecepatan pelatihan?**
    * **Teori:** Batch Normalization secara signifikan mengurangi masalah *vanishing/exploding gradients* dan membuat pelatihan lebih stabil, memungkinkan *learning rate* yang lebih besar. [cite_start]Ini seringkali mempercepat konvergensi dan menghasilkan model yang lebih baik dengan regularisasi tambahan[cite: 138, 139, 140, 144]. [cite_start]Meskipun ada overhead komputasi per *epoch*, konvergensi yang lebih cepat biasanya mengimbanginya dalam hal waktu nyata (*wall time*)[cite: 152, 153]. BN biasanya ditempatkan sebelum fungsi aktivasi.
    * **Struktur Kode:**
        ```python
        model_bn = keras.models.Sequential()
        model_bn.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
        model_bn.add(keras.layers.BatchNormalization()) # BN pertama untuk input
        for _ in range(20):
            model_bn.add(keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False)) # Hapus bias jika ada BN setelahnya
            model_bn.add(keras.layers.BatchNormalization())
            model_bn.add(keras.layers.Activation("elu"))
        model_bn.add(keras.layers.Dense(10, activation="softmax"))

        optimizer_bn = keras.optimizers.Nadam(learning_rate=0.001) # Coba LR yang sama atau lebih besar
        model_bn.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_bn, metrics=["accuracy"])

        history_bn = model_bn.fit(X_train, y_train, epochs=100,
                                   validation_data=(X_valid, y_valid),
                                   callbacks=[early_stopping_cb]) # Gunakan early_stopping yang sama
        # Plot history.history['loss'], history_bn.history['loss'] dll untuk perbandingan
        ```

    **d. Coba ganti Batch Normalization dengan SELU, dan lakukan penyesuaian yang diperlukan untuk memastikan jaringan *self-normalizes* (yaitu, standardisasi fitur input, gunakan LeCun normal initialization, pastikan DNN hanya berisi urutan lapisan *dense*, dll.).**
    * [cite_start]**Teori:** Untuk *self-normalization* dengan SELU, fitur input harus distandardisasi (mean 0, standar deviasi 1)[cite: 91]. [cite_start]Bobot harus diinisialisasi dengan LeCun normal initialization[cite: 92]. [cite_start]Jaringan harus berupa urutan lapisan *dense*[cite: 93].
    * **Struktur Kode:**
        ```python
        # Skalakan data input ke mean 0 dan std dev 1
        pixel_means = X_train_full.mean(axis=0, keepdims=True)
        pixel_stds = X_train_full.std(axis=0, keepdims=True)
        X_train_scaled = (X_train_full - pixel_means) / pixel_stds
        X_test_scaled = (X_test - pixel_means) / pixel_stds

        X_train_selu, X_valid_selu = X_train_scaled[:-5000], X_train_scaled[-5000:]
        y_train_selu, y_valid_selu = y_train_full[:-5000], y_train_full[-5000:]

        model_selu = keras.models.Sequential()
        model_selu.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
        for _ in range(20):
            model_selu.add(keras.layers.Dense(100, activation="selu",
                                              kernel_initializer="lecun_normal")) # LeCun Normal initialization
        model_selu.add(keras.layers.Dense(10, activation="softmax"))

        optimizer_selu = keras.optimizers.Nadam(learning_rate=0.001) # LR bisa tetap sama atau disesuaikan
        model_selu.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_selu, metrics=["accuracy"])

        history_selu = model_selu.fit(X_train_selu, y_train_selu, epochs=100,
                                      validation_data=(X_valid_selu, y_valid_selu),
                                      callbacks=[early_stopping_cb])
        # Bandingkan learning curves dengan model sebelumnya
        ```

    **e. Coba regularisasi model dengan *alpha dropout*. Kemudian, tanpa melatih ulang model Anda, lihat apakah Anda dapat mencapai akurasi yang lebih baik menggunakan MC Dropout.**
    * [cite_start]**Teori:** Alpha Dropout adalah varian dropout yang dirancang khusus untuk jaringan yang menggunakan SELU, karena mempertahankan properti *self-normalization*[cite: 495]. [cite_start]MC Dropout dapat meningkatkan kinerja model dropout yang sudah dilatih dan memberikan estimasi ketidakpastian yang lebih baik dengan melakukan beberapa inferensi dengan *dropout* aktif dan merata-ratakan hasilnya[cite: 497, 498].
    * **Struktur Kode:**
        ```python
        # Tambahkan Alpha Dropout ke model SELU yang sudah ada
        model_alpha_dropout = keras.models.Sequential()
        model_alpha_dropout.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
        for _ in range(20):
            model_alpha_dropout.add(keras.layers.Dense(100, activation="selu",
                                                      kernel_initializer="lecun_normal"))
            model_alpha_dropout.add(keras.layers.AlphaDropout(rate=0.1)) # Tambahkan Alpha Dropout
        model_alpha_dropout.add(keras.layers.Dense(10, activation="softmax"))

        optimizer_alpha_dropout = keras.optimizers.Nadam(learning_rate=0.001)
        model_alpha_dropout.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_alpha_dropout, metrics=["accuracy"])

        history_alpha_dropout = model_alpha_dropout.fit(X_train_selu, y_train_selu, epochs=100,
                                                        validation_data=(X_valid_selu, y_valid_selu),
                                                        callbacks=[early_stopping_cb])

        # MC Dropout (tanpa retraining)
        # Gunakan model_alpha_dropout yang sudah dilatih

        # Custom MCDropout class if other special layers are present, else direct call
        class MCAlphaDropout(keras.layers.AlphaDropout):
            def call(self, inputs):
                return super().call(inputs, training=True)

        mc_model = keras.models.Sequential()
        mc_model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
        for _ in range(20):
            mc_model.add(keras.layers.Dense(100, activation="selu",
                                            kernel_initializer="lecun_normal"))
            mc_model.add(MCAlphaDropout(rate=0.1))
        mc_model.add(keras.layers.Dense(10, activation="softmax"))

        mc_model.set_weights(model_alpha_dropout.get_weights()) # Copy weights from trained model

        n_samples = 100 # Jumlah sampel Monte Carlo
        y_probas = np.stack([mc_model(X_test_scaled, training=True) for sample in range(n_samples)])
        y_proba_mc = y_probas.mean(axis=0)
        y_pred_mc = np.argmax(y_proba_mc, axis=1)

        accuracy_mc = np.sum(y_pred_mc == y_test.flatten()) / len(y_test)
        print(f"MC Dropout Accuracy: {accuracy_mc}")
        ```

    **f. Latih ulang model Anda menggunakan 1cycle scheduling dan lihat apakah itu meningkatkan kecepatan pelatihan dan akurasi model.**
    * **Teori:** 1cycle scheduling adalah strategi *learning rate* yang melibatkan peningkatan *learning rate* secara linear di paruh pertama pelatihan dan menurunkannya di paruh kedua, seringkali dengan penyesuaian momentum. [cite_start]Ini terbukti mempercepat pelatihan dan mencapai kinerja yang lebih baik[cite: 399, 400, 403].
    * **Struktur Kode:**
        ```python
        # Untuk 1cycle scheduling, kita perlu custom callback atau menggunakan implementasi dari sumber lain
        # Contoh sederhana (implementasi 1cycle bisa kompleks)
        # Anda perlu menghitung lr_max dan lr_initial, serta menyesuaikan momentum

        class OneCycleScheduler(keras.callbacks.Callback):
            def __init__(self, lr_max, steps_per_epoch, momentum_min=0.85, momentum_max=0.95):
                self.lr_max = lr_max
                self.momentum_min = momentum_min
                self.momentum_max = momentum_max
                self.steps_per_epoch = steps_per_epoch
                self.total_steps = steps_per_epoch * self.epochs_to_run
                self.current_step = 0

            def on_train_begin(self, logs=None):
                self.epochs_to_run = self.params['epochs']
                self.lr_initial = self.lr_max / 10 # typically lr_max / 10
                keras.backend.set_value(self.model.optimizer.learning_rate, self.lr_initial)
                if hasattr(self.model.optimizer, 'momentum'):
                    keras.backend.set_value(self.model.optimizer.momentum, self.momentum_max)

            def on_train_batch_begin(self, batch, logs=None):
                self.current_step += 1
                if self.current_step <= self.total_steps / 2:
                    # Increasing LR, decreasing momentum
                    lr = self.lr_initial + (self.lr_max - self.lr_initial) * (self.current_step / (self.total_steps / 2))
                    momentum = self.momentum_max - (self.momentum_max - self.momentum_min) * (self.current_step / (self.total_steps / 2))
                else:
                    # Decreasing LR, increasing momentum
                    lr = self.lr_max - (self.lr_max - self.lr_initial) * ((self.current_step - self.total_steps / 2) / (self.total_steps / 2))
                    momentum = self.momentum_min + (self.momentum_max - self.momentum_min) * ((self.current_step - self.total_steps / 2) / (self.total_steps / 2))
                
                keras.backend.set_value(self.model.optimizer.learning_rate, lr)
                if hasattr(self.model.optimizer, 'momentum'):
                    keras.backend.set_value(self.model.optimizer.momentum, momentum)

        # Re-initialize the model (e.g., model_selu or model_bn)
        # Assuming you want to re-train the SELU model with 1cycle
        model_1cycle = keras.models.Sequential()
        model_1cycle.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
        for _ in range(20):
            model_1cycle.add(keras.layers.Dense(100, activation="selu",
                                              kernel_initializer="lecun_normal"))
            model_1cycle.add(keras.layers.AlphaDropout(rate=0.1)) # Keep alpha dropout for SELU
        model_1cycle.add(keras.layers.Dense(10, activation="softmax"))

        # Find optimal LR_max for 1cycle (similar to previous LR finding)
        # You'd typically run a LR finder for this specific model/setup first
        lr_max_for_1cycle = 0.01 # Example, find this through LR range test

        optimizer_1cycle = keras.optimizers.SGD(learning_rate=lr_max_for_1cycle/10, momentum=0.95, nesterov=True) # SGD with momentum, nesterov is common for 1cycle
        model_1cycle.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer_1cycle, metrics=["accuracy"])

        steps_per_epoch = len(X_train_selu) // 32 # Assuming batch size 32
        one_cycle_cb = OneCycleScheduler(lr_max=lr_max_for_1cycle, steps_per_epoch=steps_per_epoch)

        history_1cycle = model_1cycle.fit(X_train_selu, y_train_selu, epochs=100,
                                         validation_data=(X_valid_selu, y_valid_selu),
                                         callbacks=[early_stopping_cb, one_cycle_cb]) # Early stopping as well
        # Bandingkan hasil dengan metode sebelumnya
        ```